In [1]:
import os
import argparse
import re

import open3d as o3d
import numpy as np
from pyntcloud import PyntCloud
from multiprocessing import Pool
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Some of the functions are taken from pykitti https://github.com/utiasSTARS/pykitti/blob/master/pykitti/utils.py
def load_velo_scan(file):
    """
    Load and parse a velodyne binary file
    """
    scan = np.fromfile(file, dtype=np.float32)
    return scan.reshape((-1, 4))

def load_poses(file):
    """
    Load and parse ground truth poses
    """
    tmp_poses = np.genfromtxt(file, delimiter=' ').reshape(-1, 3, 4)
    poses = np.repeat(np.expand_dims(np.eye(4), 0), tmp_poses.shape[0], axis=0)
    poses[:, 0:3, :] = tmp_poses
    return poses

def read_calib_file(filepath):
    """
    Read in a calibration file and parse into a dictionary
    """
    data = {}

    with open(filepath, 'r') as f:
        for line in f.readlines():
            key, value = line.split(':', 1)
            try:
                data[key] = np.array([float(x) for x in value.split()])
            except ValueError:
                pass

    return data

# This part of the code is taken from the semanticKITTI API
def open_label(filename):
    """ 
    Open raw scan and fill in attributes
    """
    # check filename is string
    if not isinstance(filename, str):
        raise TypeError("Filename should be string type, "
                        "but was {type}".format(type=str(type(filename))))

    # if all goes well, open label
    label = np.fromfile(filename, dtype=np.uint32)
    label = label.reshape((-1))

    return label

def set_label(label, points):
    """ 
    Set points for label not from file but from np
    """
    # check label makes sense
    if not isinstance(label, np.ndarray):
        raise TypeError("Label should be numpy array")

    # only fill in attribute if the right size
    if label.shape[0] == points.shape[0]:
        sem_label = label & 0xFFFF  # semantic label in lower half
        inst_label = label >> 16    # instance id in upper half
    else:
        print("Points shape: ", points.shape)
        print("Label shape: ", label.shape)
        raise ValueError("Scan and Label don't contain same number of points")

    # sanity check
    assert((sem_label + (inst_label << 16) == label).all())

    return sem_label, inst_label

def transform_point_cloud(x1, R, t):
    """
    Transforms the point cloud using the giver transformation paramaters
    
    Args:
        x1  (np array): points of the point cloud [n,3]
        R   (np array): estimated rotation matrice [3,3]
        t   (np array): estimated translation vectors [3,1]
    Returns:
        x1_t (np array): points of the transformed point clouds [n,3]
    """
    x1_t = (np.matmul(R, x1.transpose()) + t).transpose()

    return x1_t

def sorted_alphanum(file_list_ordered):
    """
    Sorts the list alphanumerically
    Args:
        file_list_ordered (list): list of files to be sorted
    Return:
        sorted_list (list): input list sorted alphanumerically
    """
    def convert(text):
        return int(text) if text.isdigit() else text

    def alphanum_key(key):
        return [convert(c) for c in re.split('([0-9]+)', key)]

    sorted_list = sorted(file_list_ordered, key=alphanum_key)

    return sorted_list

def get_file_list(path, extension=None):
    """
    Build a list of all the files in the provided path
    Args:
        path (str): path to the directory 
        extension (str): only return files with this extension
    Return:
        file_list (list): list of all the files (with the provided extension) sorted alphanumerically
    """
    if extension is None:
        file_list = [os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    else:
        file_list = [
            os.path.join(path, f)
            for f in os.listdir(path)
            if os.path.isfile(os.path.join(path, f)) and os.path.splitext(f)[1] == extension
        ]
    file_list = sorted_alphanum(file_list)

    return file_list


def get_folder_list(path):
    """
    Build a list of all the folders in the provided path
    Args:
        path (str): path to the directory 
    Returns:
        folder_list (list): list of all the folders sorted alphanumerically
    """
    folder_list = [os.path.join(path, f) for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
    folder_list = sorted_alphanum(folder_list)
    
    return folder_list

In [9]:
def match_consecutive_point_cloud(pc_s, R1, t1, R2, t2):
    pc = np.matmul(np.linalg.inv(R2), (np.matmul(R1, pc_s.transpose()) + t1) - t2).transpose()
    return pc

def get_eigen_features(pc, n_neighbors=48):
    """
    Args: 
        pc (np array): points of the point cloud [n, 3]
        n_neighbors (int): number of neighbors selected
    Returns: 
        pc_feature (np array): features of the point cloud [n, 14]
    """
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pc)
    pointcloud = PyntCloud.from_instance("open3d", pcd)

    neighbors = pointcloud.get_neighbors(k=n_neighbors)
    eigenvalues = pointcloud.add_scalar_field("eigen_values", k_neighbors=neighbors)

    anisotropy = pointcloud.add_scalar_field("anisotropy", ev=eigenvalues)
    curvature = pointcloud.add_scalar_field("curvature", ev=eigenvalues)
    eigenentropy = pointcloud.add_scalar_field("eigenentropy", ev=eigenvalues)
    eigensum = pointcloud.add_scalar_field("eigen_sum", ev=eigenvalues)
    linearity = pointcloud.add_scalar_field("linearity", ev=eigenvalues)
    omnivariance = pointcloud.add_scalar_field("omnivariance", ev=eigenvalues)
    planarity = pointcloud.add_scalar_field("planarity", ev=eigenvalues)
    sphericity = pointcloud.add_scalar_field("sphericity", ev=eigenvalues)

    pc_feature = np.asarray(pointcloud.points)

    return pc_feature

def cal_chamfer_dist(source_pts, target_pts, length):
    if(target_pts.size == 0):
        return 24 * length * length
    source_pcd = o3d.geometry.PointCloud()
    source_pcd.points = o3d.utility.Vector3dVector(source_pts[:, 0:3].reshape(-1, 3))
    target_pcd = o3d.geometry.PointCloud()
    target_pcd.points = o3d.utility.Vector3dVector(target_pts[:, 0:3].reshape(-1, 3))
    
    source2target_dists = source_pcd.compute_point_cloud_distance(target_pcd)
    target2source_dists = target_pcd.compute_point_cloud_distance(source_pcd)
    source2target_dists = np.asarray(source2target_dists)
    target2source_dists = np.asarray(target2source_dists)
    source2target_dists = np.square(source2target_dists)
    target2source_dists = np.square(target2source_dists)
    chamfer_dist = source2target_dists.mean() + target2source_dists.mean()
    
    return chamfer_dist

def find_neighbors(voxel_center, source_pts, target_pts, length):
    source_idx = np.where((source_pts[:,0] > voxel_center[0] - length) & (source_pts[:,0] < voxel_center[0] + length) & 
                          (source_pts[:,1] > voxel_center[1] - length) & (source_pts[:,1] < voxel_center[1] + length) & 
                          (source_pts[:,2] > voxel_center[2] - length) & (source_pts[:,2] < voxel_center[2] + length))[0]
    target_idx = np.where((target_pts[:,0] > voxel_center[0] - length) & (target_pts[:,0] < voxel_center[0] + length) & 
                          (target_pts[:,1] > voxel_center[1] - length) & (target_pts[:,1] < voxel_center[1] + length) & 
                          (target_pts[:,2] > voxel_center[2] - length) & (target_pts[:,2] < voxel_center[2] + length))[0]
    
    source_pts = source_pts[source_idx, :]
    target_pts = target_pts[target_idx, :]
    
    return source_pts, target_pts, source_idx

def chamfer_dist(source_pts, target_pts, length=1.5):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(source_pts)
    voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=length * 2)
    voxels_all = voxel_grid.get_voxels()
    
    chamfer = np.ones(source_pts.shape[0]) * -1
    
    for voxel in voxels_all:
        voxel_center = voxel_grid.get_voxel_center_coordinate(voxel.grid_index)
        source_neighbors_pts, target_neighbors_pts, source_neighbors_idx = find_neighbors(voxel_center, source_pts, target_pts, length)
        chamfer[source_neighbors_idx] = cal_chamfer_dist(source_neighbors_pts, target_neighbors_pts, length)
    
    return chamfer

In [4]:
def get_transformation_result(pc, Rt):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pc.reshape(-1, 3))
    pcd.transform(Rt)
    
    return np.asarray(pcd.points)

def plot_consecutive_point_cloud(pc_s, pc_t):
    pcd_s = o3d.geometry.PointCloud()
    pcd_s.points = o3d.utility.Vector3dVector(pc_s[:, 0:3])
    pcd_s.paint_uniform_color([1, 0, 0])

    pcd_t = o3d.geometry.PointCloud()
    pcd_t.points = o3d.utility.Vector3dVector(pc_t[:, 0:3])
    pcd_t.paint_uniform_color([0, 1, 0])

    o3d.visualization.draw_geometries([pcd_s, pcd_t])

In [10]:
file_list = get_file_list('\\Workspace\\SceneFlow\\datasets\\stereo_kitti')
poses = load_poses('\\Workspace\\SceneFlow\\lib\\stereoKITTI2.txt')

for idx, file in enumerate(file_list):
#     if idx < 150:
#         continue
    pc_data = np.load(file)
    pc_s = pc_data['pc1']
    pc_t = pc_data['pc2']
    flow_gt = pc_data['flow']
    
    pc_s_w = get_transformation_result(pc_s, poses[idx])
    flow_estimate = pc_s_w - pc_s
    
#     pc_s[:, [1, 2]] = pc_s[:, [2, 1]]
#     pc_t[:, [1, 2]] = pc_t[:, [2, 1]]
    
#     R1 = np.eye(3)
#     t1 = np.zeros((3, 1))
#     R2 = poses[idx, 0:3, 0:3]
#     t2 = poses[idx, 0:3, 3].reshape(3, 1)

#     def match_consecutive_point_cloud(pc_s, R1, t1, R2, t2):
#         pc = np.matmul(np.linalg.inv(R2), (np.matmul(R1, pc_s.transpose()) + t1) - t2).transpose()
#         return pc

#     pc_s = match_consecutive_point_cloud(pc_s, R1, t1, R2, t2)
#     plot_consecutive_point_cloud(pc_s_w, pc_t)
    
    # Extract eigen features
    eigen_features_s = get_eigen_features(pc_s_w)
    eigen_features_t = get_eigen_features(pc_t)
    
    # Calculate Chamfer distance
    dist = np.array(chamfer_dist(pc_s_w, pc_t))
    
    n1 = pc_s_w.shape[0]
    n2 = pc_t.shape[0]
    if n1 >= 16384:
        sample_idx1 = np.random.choice(n1, 16384, replace=False)
    else:
        sample_idx1 = np.concatenate((np.arange(n1), np.random.choice(n1, 16384 - n1, replace=True)), axis=-1)
    if n2 >= 16384:
        sample_idx2 = np.random.choice(n2, 16384, replace=False)
    else:
        sample_idx2 = np.concatenate((np.arange(n2), np.random.choice(n2, 16384 - n2, replace=True)), axis=-1)
        
    pc_s_w = pc_s_w[sample_idx1, :]
    pc_t = pc_t[sample_idx2, :]
    eigen_features_s = eigen_features_s[sample_idx1, :]
    eigen_features_t = eigen_features_t[sample_idx2, :]
    dist = dist[sample_idx1]
    flow_gt = flow_gt[sample_idx1, :]
    flow_estimate = flow_estimate[sample_idx1, :]
    
#     plot_consecutive_point_cloud(pc_s, pc_t)

    np.savez_compressed(os.path.join('\\Workspace\\SceneFlow\\datasets', 'stereoKITTI_dynamic_estimation_3', '{}.npz'.format(str(idx).zfill(6))),
                        pc1=pc_s_w,
                        pc2=pc_t,
                        eigen_features_s=eigen_features_s,
                        eigen_features_t=eigen_features_t,
                        chamfer_dist=dist,
                        flow_gt=flow_gt,
                        flow_estimate=flow_estimate)
    print(idx)
#     break

0
1
2
3
4


D:\jiahaogu\anaconda3\lib\site-packages\pyntcloud\scalar_fields\eigenvalues.py:50: RuntimeWarning: divide by zero encountered in log
  result += ev[:, i] * np.log(ev[:, i])
D:\jiahaogu\anaconda3\lib\site-packages\pyntcloud\scalar_fields\eigenvalues.py:50: RuntimeWarning: invalid value encountered in multiply
  result += ev[:, i] * np.log(ev[:, i])


5
6
7
8
9
10
11
12


D:\jiahaogu\anaconda3\lib\site-packages\pyntcloud\scalar_fields\eigenvalues.py:50: RuntimeWarning: divide by zero encountered in log
  result += ev[:, i] * np.log(ev[:, i])
D:\jiahaogu\anaconda3\lib\site-packages\pyntcloud\scalar_fields\eigenvalues.py:50: RuntimeWarning: invalid value encountered in multiply
  result += ev[:, i] * np.log(ev[:, i])


13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


...

41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108


D:\jiahaogu\anaconda3\lib\site-packages\pyntcloud\scalar_fields\eigenvalues.py:50: RuntimeWarning: invalid value encountered in log
  result += ev[:, i] * np.log(ev[:, i])
D:\jiahaogu\anaconda3\lib\site-packages\pyntcloud\scalar_fields\eigenvalues.py:77: RuntimeWarning: invalid value encountered in power
  self.to_be_added[name] = np.nan_to_num((ev[:, 0] * ev[:, 1] * ev[:, 2]) ** (1 / 3))
D:\jiahaogu\anaconda3\lib\site-packages\pyntcloud\scalar_fields\eigenvalues.py:50: RuntimeWarning: divide by zero encountered in log
  result += ev[:, i] * np.log(ev[:, i])
D:\jiahaogu\anaconda3\lib\site-packages\pyntcloud\scalar_fields\eigenvalues.py:50: RuntimeWarning: invalid value encountered in multiply
  result += ev[:, i] * np.log(ev[:, i])


109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143


D:\jiahaogu\anaconda3\lib\site-packages\pyntcloud\scalar_fields\eigenvalues.py:50: RuntimeWarning: divide by zero encountered in log
  result += ev[:, i] * np.log(ev[:, i])
D:\jiahaogu\anaconda3\lib\site-packages\pyntcloud\scalar_fields\eigenvalues.py:50: RuntimeWarning: invalid value encountered in multiply
  result += ev[:, i] * np.log(ev[:, i])
D:\jiahaogu\anaconda3\lib\site-packages\pyntcloud\scalar_fields\eigenvalues.py:50: RuntimeWarning: divide by zero encountered in log
  result += ev[:, i] * np.log(ev[:, i])
D:\jiahaogu\anaconda3\lib\site-packages\pyntcloud\scalar_fields\eigenvalues.py:50: RuntimeWarning: invalid value encountered in multiply
  result += ev[:, i] * np.log(ev[:, i])


144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
